In [5]:
from sklearn import tree
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef, balanced_accuracy_score
import time

In [ ]:
# load ROOT
import ROOT

In [ ]:
# load TNeutron class
ROOT.gSystem.Load('/home/user/data/ML/lib/v6_30/TNeutron_cc.so')

In [8]:
# set the path to our simulation file
#datapath = "/home/user/data/ML/MLP_mult2/"
#datapath = "/home/user/data/ML/e15118/"
datapath = "/home/user/data/ML/stmona/"

In [9]:
# open the ROOT file and load the TTree that it contains
#rootfile = ROOT.TFile(datapath + "delta18_O262plus_2nThermal2.root","READ")
# try this one for training
rootfile = ROOT.TFile(datapath + "tneutron_mult2_26O_24O+2n-uniform.root","READ")
# try this one to test your trained model
rootfile1 = ROOT.TFile(datapath + "tneutron_mult2_26O_24O+2n-delta500keV.root","READ")
tre1= rootfile1.Get("snt")
tre = rootfile.Get("snt")
#tree.Print()

In [10]:
# copy data from the tree
entries = tre.GetEntries();
hit_data = np.zeros((entries, 10));
label = np.empty(entries);
label.fill(9);
for i in range(entries):
  # get the tree index for the next entry on our list
  # and load this entry from the TTree
  tre.GetEntry(i)
  # transfer the data for this entry to our numpy array
  # x0
  hit_data[i][0] = tre.g.x[0]
  # y0
  hit_data[i][1] = tre.g.y[0]
  # z0
  hit_data[i][2] = tre.g.z[0]
  # t0
  hit_data[i][3] = tre.g.t[0]
  # q0
  hit_data[i][4] = tre.g.q[0]
  # x1
  hit_data[i][5] = tre.g.x[1]
  # y1
  hit_data[i][6] = tre.g.y[1]
  # z1
  hit_data[i][7] = tre.g.z[1]
  # t1
  hit_data[i][8] = tre.g.t[1]
  # q1
  hit_data[i][9] = tre.g.q[1]
  # label
  label[i]= tre.signal
  #label[i]= tree.target

In [11]:
# create calculated data matrix
entries = tre.GetEntries();
hit_calc= np.zeros((entries, 7));
for i in range(entries):
  # calculated quantities
  tre.GetEntry(i);
  hit_calc[i][0] = tre.g.GetVelocity(0);
  hit_calc[i][1] = tre.g.GetVelocity(1);
  hit_calc[i][2] = tre.g.HitSeparation(0,1);
  hit_calc[i][3] = tre.g.HitVdiff(0,1);
  hit_calc[i][4] = tre.g.HitOpeningAngle(0,1);
  hit_calc[i][5] = tre.g.HitNSI(0,1);
  hit_calc[i][6] = tre.g.HitScatteringAngle(0,1);

In [12]:
# copy data from the tree
entries1 = tre1.GetEntries();
hit_data1 = np.zeros((entries1, 10));
label1 = np.empty(entries1);
label1.fill(9);
for o in range(entries1):
  # get the tree index for the next entry on our list
  # and load this entry from the TTree
  tre1.GetEntry(o)
  # transfer the data for this entry to our numpy array
  # x0
  hit_data1[o][0] = tre1.g.x[0]
  # y0
  hit_data1[o][1] = tre1.g.y[0]
  # z0
  hit_data1[o][2] = tre1.g.z[0]
  # t0
  hit_data1[o][3] = tre1.g.t[0]
  # q0
  hit_data1[o][4] = tre1.g.q[0]
  # x1
  hit_data1[o][5] = tre1.g.x[1]
  # y1
  hit_data1[o][6] = tre1.g.y[1]
  # z1
  hit_data1[o][7] = tre1.g.z[1]
  # t1
  hit_data1[o][8] = tre1.g.t[1]
  # q1
  hit_data1[o][9] = tre1.g.q[1]
  # label
  label1[o]= tre1.signal

In [13]:
# create calculated data matrix
entries1 = tre1.GetEntries();
hit_calc1= np.zeros((entries1, 7));
for o in range(entries1):
  # calculated quantities
  tre1.GetEntry(o);
  hit_calc1[o][0] = tre1.g.GetVelocity(0);
  hit_calc1[o][1] = tre1.g.GetVelocity(1);
  hit_calc1[o][2] = tre1.g.HitSeparation(0,1);
  hit_calc1[o][3] = tre1.g.HitVdiff(0,1);
  hit_calc1[o][4] = tre1.g.HitOpeningAngle(0,1);
  hit_calc1[o][5] = tre1.g.HitNSI(0,1);
  hit_calc1[o][6] = tre1.g.HitScatteringAngle(0,1);

In [14]:
# Concatenate measured and calculated quantities
input_data_min_max1 = np.concatenate((hit_data1, hit_calc1), axis=1)
X1 = input_data_min_max1
y1 = label1
# Scale the features using Standered
scaler = StandardScaler()
Dataset_featured1 = scaler.fit_transform(X1)

In [15]:
# Concatenate measured and calculated quantities
input_data_min_max = np.concatenate((hit_data, hit_calc), axis=1)
X = input_data_min_max
y = label

# Scale the features using Standered
Dataset_featured = scaler.fit_transform(X)

In [16]:
clf = tree.DecisionTreeClassifier()

In [ ]:
number = []  # To store the number of data points (i.e., sample size in each iteration)
timearray = []  # To store the time taken for each iteration
accuracy_list = []  # To store the accuracy score for each iteration
bas_list = []  # To store the balanced accuracy score for each iteration
F1_list = []  # To store the F1 score for each iteration
Mc_list = []  # To store the Matthews Correlation Coefficient for each iteration

# Loop over different sample sizes starting from 100 to 20000 with a step of 500
for i in range(100, 20000, 500):
    start = time.time()  # Start timer

    # Select the first i samples from Dataset_featured and corresponding labels
    selection_X1 = Dataset_featured[0:i, :]
    selection_Y1 = y[0:i]

    # Train classifier clf with the selected samples
    clf.fit(selection_X1, selection_Y1)

    # Predict labels using the fitted model on Dataset_featured1 (test data presumably)
    y_pred = clf.predict(Dataset_featured1)

    end = time.time()  # End timer
    length = end - start  # Calculate elapsed time

    # Print the unique values of true labels (y1) and predicted labels (y_pred)
    print(set(y1))
    print(set(y_pred))

    # Convert y1 and y_pred to pandas Series to explore unique values
    y1_series = pd.Series(y1)
    y_pred_series = pd.Series(y_pred)
    print(y1_series.unique())
    print(y_pred_series.unique())

    # Calculate various metrics between the true labels (y1) and predicted labels (y_pred)
    accuracy = accuracy_score(y1, y_pred)  # Accuracy score
    Mc = matthews_corrcoef(y1, y_pred)  # Matthews correlation coefficient
    F1 = f1_score(y1, y_pred, average='micro')  # F1 score with 'micro' averaging
    bas = balanced_accuracy_score(y1, y_pred)  # Balanced accuracy score

    # Append computed metrics and time to corresponding lists
    timearray.append(length)  # Time taken
    number.append(i)  # Number of samples used
    accuracy_list.append(accuracy)  # Accuracy score
    bas_list.append(bas)  # Balanced accuracy score
    F1_list.append(F1)  # F1 score
    Mc_list.append(Mc)  # Matthews correlation coefficient

    # Print results of the current iteration
    print("Accuracy:", accuracy)
    print("It took", length, "seconds!", i)

# Print the lists containing results after all iterations
print(number)
print(timearray)
print("Accuracy:", accuracy_list)
print("F1 Accuracy:", F1_list)
print("bas Accuracy:", bas_list)
print("Mc Accuracy:", Mc_list)

# Plot the metrics against the number of data points used
plt.plot(number, accuracy_list, label="accuracy")  # Plot accuracy
plt.plot(number, F1_list, label="F1")  # Plot F1 score
plt.plot(number, bas_list, label="bas")  # Plot balanced accuracy
plt.plot(number, Mc_list, label="Mc")  # Plot Matthews correlation coefficient

# Label the axes and give the plot a title
plt.xlabel('number of data points')
plt.ylabel('accuracy')
plt.title('Accuracy')
plt.legend()  # Display the legend


In [37]:
feature_names=["x0","y0","z0","t0","q0","x1","y1","z1","t1","q1","Velocity","Velocity1","HitSep","HitVdiff","HOA","HNSI","HSA"]
labels = ["background","signal"]

In [ ]:
plt.figure(figsize=(30,10), facecolor ='k')

a = tree.plot_tree(clf,feature_names = feature_names, class_names =labels, rounded = True, filled = True, fontsize=14)

plt.show()


from sklearn.tree import export_text

tree_rules = export_text(clf,feature_names = list(feature_names))

print(tree_rules)

In [ ]:
import seaborn as sns
from sklearn import metrics

confusion_matrix = metrics.confusion_matrix(y1, y_pred)

matrix_df = pd.DataFrame(confusion_matrix)

ax = plt.axes()

sns.set(font_scale=1.3)

plt.figure(figsize=(10,7))

sns.heatmap(matrix_df, annot=True, fmt="g", ax=ax, cmap="magma")

ax.set_title('Confusion Matrix - Decision Tree')

ax.set_xlabel("Predicted label", fontsize =15)

ax.set_xticklabels([]+labels)

ax.set_ylabel("True Label", fontsize=15)

ax.set_yticklabels(list(labels), rotation = 0)

plt.show()